In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
%pip install semantic-link-labs

In [ ]:
import sempy_labs as labs
import sempy_labs.report as rep
import sempy.fabric as fabric
from sempy_labs import admin
from datetime import datetime, timedelta
import pandas as pd

admin.list_tenant_settings()

In [ ]:
# Test to make sure that activity events are working

today = datetime.utcnow().strftime("%Y-%m-%d")
start_time = f"{today}T00:00:00"
end_time = f"{today}T23:59:59"


df = admin.list_activity_events(start_time=start_time, end_time=end_time, activity_filter=None)
display(df)


In [ ]:
# Loop through last 30 days

# 🔧 Config
days_back = 30  # how many days of history to pull
activity_filter = "viewreport"
all_logs = []

# 🕒 Loop through one day at a time
for i in range(days_back):
    day = datetime.utcnow() - timedelta(days=i)
    start_time = f"{day.strftime('%Y-%m-%d')}T00:00:00"
    end_time = f"{day.strftime('%Y-%m-%d')}T23:59:59"

    try:
        df_day = admin.list_activity_events(
            start_time=start_time,
            end_time=end_time,
            activity_filter=activity_filter
        )
        if not df_day.empty:
            print(f"✅ Data for {day.strftime('%Y-%m-%d')}: {len(df_day)} rows")
            all_logs.append(df_day)
        else:
            print(f"⚠️ No activity on {day.strftime('%Y-%m-%d')}")
    except Exception as e:
        print(f"❌ Error on {day.strftime('%Y-%m-%d')}: {e}")

# 📦 Combine results into single DataFrame
if all_logs:
    df_usage = pd.concat(all_logs, ignore_index=True)
    print(f"\n✅ Combined total: {len(df_usage)} rows")
    display(df_usage.head())
else:
    print("⚠️ No usage data collected.")


In [ ]:
# List all workspaces you have access to
df_workspaces = fabric.list_workspaces()
df_workspaces.head()

In [ ]:
#Store all reports across workspaces
all_reports = []

# Loop through each workspace ID
for _, row in df_workspaces.iterrows():
    ws_id = row["Id"]
    ws_name = row["Name"]

    try:
        df_reports = fabric.list_reports(workspace=ws_id)
        if not df_reports.empty:
            df_reports["Workspace Name"] = ws_name  # tag workspace name
            all_reports.append(df_reports)
            print(f"✅ Reports found in: {ws_name}")
        else:
            print(f"⚠️ No reports in: {ws_name}")
    except Exception as e:
        print(f"❌ Error retrieving from {ws_name}: {e}")

if all_reports:
    df_all_reports = pd.concat(all_reports, ignore_index=True)
    display(df_all_reports.head())
else:
    print("⚠️ No reports found in any workspace.")

display(df_reports.head())

In [ ]:
# Create a list to hold each workspace's reports
all_reports = []

# Loop through each workspace and append report data
for ws_id, ws_name in zip(df_workspaces["Id"], df_workspaces["Name"]):
    df_reports = fabric.list_reports(workspace=ws_id)
    if not df_reports.empty:
        df_reports["WorkspaceId"] = ws_id
        df_reports["WorkspaceName"] = ws_name
        all_reports.append(df_reports)

# Concatenate into one DataFrame
df_all_reports = pd.concat(all_reports, ignore_index=True)
df_all_reports = df_all_reports[["Name", "WorkspaceName"]]


# Display the combined DataFrame
display(df_all_reports.head())

In [ ]:
df_spark = spark.createDataFrame(df_all_reports)
df_spark.write.mode("overwrite").format("delta").saveAsTable("AdminReports")

In [ ]:
# 🗓️ Pull usage for the last 30 days
days_back = 90
activity_filter = "viewreport"
usage_logs = []

for i in range(days_back):
    date = datetime.utcnow() - timedelta(days=i)
    start_time = f"{date.strftime('%Y-%m-%d')}T00:00:00"
    end_time = f"{date.strftime('%Y-%m-%d')}T23:59:59"

    try:
        df_day = admin.list_activity_events(
            start_time=start_time,
            end_time=end_time,
            activity_filter=activity_filter
        )
        usage_logs.append(df_day)
    except:
        continue

df_usage = pd.concat(usage_logs, ignore_index=True)


In [ ]:
df_usage.head()

In [ ]:
# Save to csv
df_usage.to_csv('/lakehouse/default/' + 'Files/df_usage.csv', index=False)
df_usage.columns= [col.replace(" ", "_") for col in df_usage.columns]

In [ ]:
df_spark = spark.createDataFrame(df_usage)
df_spark.write.mode("overwrite").format("delta").save("Tables/Usage")


In [ ]:
df2 = spark.read.format("delta").load("Tables/Usage")
# Convert and preview first 5 rows
df2.toPandas().head(5)
df2.toPandas().shape

https://github.com/microsoft/semantic-link-labs

https://tabulareditor.com/learn